In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/real-test-data/dataset/sample_test.csv
/kaggle/input/real-test-data/dataset/sample_test_out_fail.csv
/kaggle/input/real-test-data/dataset/sample_test_out.csv
/kaggle/input/real-test-data/dataset/train.csv
/kaggle/input/real-test-data/dataset/test.csv


# OUTLINE 
* Load the image dataset.
* Apply OCR using EasyOCR with preprocessing (e.g., grayscale conversion, thresholding).
* Clean common OCR mistakes.
* Generate a final DataFrame with metadata.

In [4]:
df = pd.read_csv("/kaggle/input/real-test-data/dataset/test.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131187 entries, 0 to 131186
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   index        131187 non-null  int64 
 1   image_link   131187 non-null  object
 2   group_id     131187 non-null  int64 
 3   entity_name  131187 non-null  object
dtypes: int64(2), object(2)
memory usage: 4.0+ MB


# Key Fix:
* Convert Pillow Image to Numpy Array: In the perform_ocr() function, the grayscale Pillow image is now converted to a numpy array using np.array(preprocessed_image) before passing it to EasyOCR

In [6]:
import requests
from PIL import Image, ImageOps
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import easyocr

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)

# Sample image URL
image_url = "https://m.media-amazon.com/images/I/81bLHedrwL._AC_SL1500_.jpg"

# Function to preprocess and display the image
def preprocess_image(image_url):
    try:
        # Send a request to get the image from the URL
        response = requests.get(image_url)
        response.raise_for_status()  # Ensure the request was successful
        
        # Open the image and convert it to grayscale
        img = Image.open(BytesIO(response.content))
        gray_img = ImageOps.grayscale(img)
        
        # Display the image
        plt.imshow(gray_img, cmap='gray')
        plt.axis('off')  # Turn off the axis
        plt.show()
        
        return gray_img
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {image_url}: {e}")
        return None

# Preprocess and display the image
preprocessed_image = preprocess_image(image_url)

# Perform OCR if the image was processed successfully
if preprocessed_image:
    # Convert Pillow image to numpy array
    np_image = np.array(preprocessed_image)
    
    # Perform OCR using EasyOCR
    result = reader.readtext(np_image, detail=0)  # detail=0 returns only the text
    extracted_text = " ".join(result)  # Join the OCR result into a single string
    print(extracted_text)
else:
    print("Error processing image")


/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Error downloading image from https://m.media-amazon.com/images/I/81bLHedrwL._AC_SL1500_.jpg: 400 Client Error: Bad Request for url: https://m.media-amazon.com/images/I/81bLHedrwL._AC_SL1500_.jpg
Error processing image


In [7]:
import re

def extract_weight(text):
    # Regular expression pattern to match item weights (e.g., "90 lb", "160gms", "160 grams")
    weight_pattern = r"(\d+)\s?(lb|gms|grams|kg|kilograms)"
    
    # Search for all matches in the text
    matches = re.findall(weight_pattern, text, re.IGNORECASE)
    
    # Prepare a list to store formatted weights
    weights = []
    
    # Process each match to clean up the format
    for match in matches:
        weight_value, weight_unit = match
        weight_value = weight_value.strip()  # Remove extra spaces
        weight_unit = weight_unit.lower()    # Normalize units to lower case
        weights.append(f"{weight_value} {weight_unit}")

    # Return list of extracted weights
    return weights

# Example usage with series of text data
texts = [
    "sfdf height fksdj weight hdf 76 lbs 22lbs" ]

# Iterate over texts and extract weights
for i, text in enumerate(texts, start=1):
    extracted_weights = extract_weight(text)
    print(f"Text {i} - Extracted Weights: {extracted_weights}")


Text 1 - Extracted Weights: ['76 lb', '22 lb']


# BASIC PREPROCESSING
* Sort the CSV by the entity_name column alphabetically.
* Create separate dataframes for each unique entity_name.
* Ensure that the total number of rows remains intact across all separate dataframes.

In [8]:
import pandas as pd

# Load the CSV data into a pandas DataFrame (replace 'your_csv_file.csv' with your actual CSV file path)
data = pd.read_csv('your_csv_file.csv')

# Step 1: Sort the dataframe by the 'entity_name' column alphabetically
sorted_data = data.sort_values(by='entity_name')

# Step 2: Create separate dataframes for each unique 'entity_name'
grouped_dfs = {entity: sorted_data[sorted_data['entity_name'] == entity] for entity in sorted_data['entity_name'].unique()}

# Step 3: Verify the total number of rows in all separate dataframes is equal to the original dataframe
total_rows = sum([len(df) for df in grouped_dfs.values()])
original_rows = len(sorted_data)

# Check if the number of rows is intact
if total_rows == original_rows:
    print("The total number of rows is intact!")
else:
    print(f"Row mismatch: {total_rows} rows in separate dataframes, {original_rows} rows in original dataframe.")

# Optional: Display or use the separate dataframes
for entity, df in grouped_dfs.items():
    print(f"Dataframe for entity '{entity}':")
    print(df, "\n")


FileNotFoundError: [Errno 2] No such file or directory: 'your_csv_file.csv'

# Load the CSV data into a pandas DataFrame (replace 'your_csv_file.csv' with your actual CSV file path)

In [10]:
data = pd.read_csv('/kaggle/input/real-test-data/dataset/test.csv')

# Step 1: Sort the dataframe by the 'entity_name' column alphabetically

In [11]:
sorted_data = data.sort_values(by='entity_name')
sorted_data.head()

,index,image_link,group_id,entity_name
47896,47943,https://m.media-amazon.com/images/I/51Tj2jPW0v...,296275,depth
38494,38538,https://m.media-amazon.com/images/I/51JU5+OZWN...,648011,depth
21600,21630,https://m.media-amazon.com/images/I/510y3mbpLx...,866516,depth
60216,60268,https://m.media-amazon.com/images/I/51hE0fs44f...,488964,depth
21598,21628,https://m.media-amazon.com/images/I/510y02x7cM...,453674,depth



# Step 2: Create separate dataframes for each unique 'entity_name'

In [12]:
grouped_dfs = {entity: sorted_data[sorted_data['entity_name'] == entity] for entity in sorted_data['entity_name'].unique()}

# Step 3: Verify the total number of rows in all separate dataframes is equal to the original dataframe

In [13]:
total_rows = sum([len(df) for df in grouped_dfs.values()])
original_rows = len(sorted_data)
print(total_rows)
print(original_rows)

131187
131187


# Check if the number of rows is intact

In [14]:
if total_rows == original_rows:
    print("The total number of rows is intact!")
else:
    print(f"Row mismatch: {total_rows} rows in separate dataframes, {original_rows} rows in original dataframe.")

The total number of rows is intact!


# Optional: Display or use the separate dataframes

In [15]:
for entity, df in grouped_dfs.items():
    print(f"Dataframe for entity '{entity}':")
    print(df, "\n")


Dataframe for entity 'depth':
       index                                         image_link  group_id  \
47896  47943  https://m.media-amazon.com/images/I/51Tj2jPW0v...    296275   
38494  38538  https://m.media-amazon.com/images/I/51JU5+OZWN...    648011   
21600  21630  https://m.media-amazon.com/images/I/510y3mbpLx...    866516   
60216  60268  https://m.media-amazon.com/images/I/51hE0fs44f...    488964   
21598  21628  https://m.media-amazon.com/images/I/510y02x7cM...    453674   
...      ...                                                ...       ...   
9317    9328  https://m.media-amazon.com/images/I/41Qqaynypv...    752266   
94549  94623  https://m.media-amazon.com/images/I/618CGovZ6i...    529606   
94534  94608  https://m.media-amazon.com/images/I/618Bl445n5...    430434   
54179  54226  https://m.media-amazon.com/images/I/51aZrdA52Z...    927393   
10953  10965  https://m.media-amazon.com/images/I/41Wic7r39W...    594224   

      entity_name  
47896       depth  
38494

# Save each dataframe to a separate CSV file

In [16]:

for entity, df in grouped_dfs.items():
    # Create a valid file name (remove any special characters or spaces)
    file_name = f"{entity}_data.csv".replace(" ", "_")
    
    # Save the dataframe as a CSV file
    df.to_csv(file_name, index=False)
    print(f"Saved dataframe for entity '{entity}' to '{file_name}'")

Saved dataframe for entity 'depth' to 'depth_data.csv'
Saved dataframe for entity 'height' to 'height_data.csv'
Saved dataframe for entity 'item_volume' to 'item_volume_data.csv'
Saved dataframe for entity 'item_weight' to 'item_weight_data.csv'
Saved dataframe for entity 'maximum_weight_recommendation' to 'maximum_weight_recommendation_data.csv'
Saved dataframe for entity 'voltage' to 'voltage_data.csv'
Saved dataframe for entity 'wattage' to 'wattage_data.csv'
Saved dataframe for entity 'width' to 'width_data.csv'


# 1)Wattage
# 2)Volatge
# 3)Maximum Weight Recommendation
# 4)Item volume

In [17]:
import pandas as pd

# Load the 'depth_data.csv' file
item_volume_df = pd.read_csv('item_volume_data.csv')

# Display the top 5 rows of the 'depth_data.csv'
print(item_volume_df.head(10))


    index                                         image_link  group_id  \
0  128522  https://m.media-amazon.com/images/I/81bgTNB4g-...    257505   
1  128501  https://m.media-amazon.com/images/I/81bLHedrwL...    426449   
2  115029  https://m.media-amazon.com/images/I/711dIDwDuJ...    142295   
3   41358  https://m.media-amazon.com/images/I/51MVfM5NJV...    588284   
4  112194  https://m.media-amazon.com/images/I/61jCNOENUw...    784861   
5  127237  https://m.media-amazon.com/images/I/81KSiBGcmW...    589528   
6  114978  https://m.media-amazon.com/images/I/711EbPu-Nh...    417442   
7  127232  https://m.media-amazon.com/images/I/81KOZGbVZB...    125148   
8  127456  https://m.media-amazon.com/images/I/81N6CnmBpr...    162326   
9  112191  https://m.media-amazon.com/images/I/61j9aDDdjD...    472418   

   entity_name  
0  item_volume  
1  item_volume  
2  item_volume  
3  item_volume  
4  item_volume  
5  item_volume  
6  item_volume  
7  item_volume  
8  item_volume  
9  item_volume 

In [18]:
# Display rows from index 10 to 20 (note that iloc is zero-indexed)
print(item_volume_df.iloc[10:21])  # 10th to 20th rows, including row at index 20


     index                                         image_link  group_id  \
10   45081  https://m.media-amazon.com/images/I/51Qb3xsnqj...    961822   
11  127477  https://m.media-amazon.com/images/I/81NLC-MbzN...    911827   
12  111793  https://m.media-amazon.com/images/I/61g-R0dUWv...    373285   
13  112234  https://m.media-amazon.com/images/I/61jWtTCGOF...    588284   
14  114989  https://m.media-amazon.com/images/I/711JmTEgBU...    692827   
15  128678  https://m.media-amazon.com/images/I/81dup5ucjW...    976920   
16  111771  https://m.media-amazon.com/images/I/61fwkjqJ0m...    941694   
17  121564  https://m.media-amazon.com/images/I/71cjjOrq97...    142295   
18  111742  https://m.media-amazon.com/images/I/61fn0eiL6F...    287137   
19   63256  https://m.media-amazon.com/images/I/51kWTG56Jd...    941694   
20  112264  https://m.media-amazon.com/images/I/61jnmb4d4J...    693967   

    entity_name  
10  item_volume  
11  item_volume  
12  item_volume  
13  item_volume  
14  item_

# PROCESS FOR ITEM_VOLUME(3833 ROWS)

*  We can loop through each row of your item_volume dataframe,
* download the image using the image_link, perform OCR on the image to extract the text, 
* and then extract only the value + unit (like "5 liters", "12 cm³", etc.).
* If no value + unit is found in the OCR result, we will return a blank (" ").

In [20]:
import requests
from PIL import Image, ImageOps
from io import BytesIO
import numpy as np
import pandas as pd
import easyocr
import re

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)

# Function to preprocess and perform OCR on the image URL
def extract_value_unit_from_image(image_url):
    try:
        # Send a request to get the image from the URL
        response = requests.get(image_url)
        response.raise_for_status()  # Ensure the request was successful

        # Open the image and convert it to grayscale
        img = Image.open(BytesIO(response.content))
        gray_img = ImageOps.grayscale(img)
        
        # Convert Pillow image to numpy array for OCR
        np_image = np.array(gray_img)
        
        # Perform OCR using EasyOCR
        result = reader.readtext(np_image, detail=0)  # detail=0 returns only the text
        extracted_text = " ".join(result)  # Join the OCR result into a single string
        
        # Use regex to find the value + unit (e.g., numbers followed by units like cm³, L, etc.)
        pattern = r'(\d+\.?\d*)\s?(cm³|liters|ml|g|kg|m³|in³|L|oz|fl oz)'  # Modify as per expected units
        matches = re.findall(pattern, extracted_text)
        pattern = r'(\d+)\s*(mL)'
        matches = re.findall(pattern,extracted_text)

        if matches:
            # Return the first match (value + unit)
            return f"{matches[0][0]} {matches[0][1]}"
        else:
            # Return blank if no match is found
            return " "
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {image_url}: {e}")
        return " "  # Return blank in case of an error

# Assuming you already have the item_volume dataframe (df_item_volume)
df_item_volume = pd.DataFrame({
    'index': [128522, 128501, 115029, 41358, 112194],
    'image_link': [
        "https://m.media-amazon.com/images/I/81bgTNB4g-L._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/81bLHedrwL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/711dIDwDuJ._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/51MVfM5NJV._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61jCNOENUwL._AC_SL1500_.jpg"
    ],
    'group_id': [257505, 426449, 142295, 588284, 784861],
    'entity_name': ['item_volume', 'item_volume', 'item_volume', 'item_volume', 'item_volume']
})

# Create a new column to store the extracted item volume (value + unit)
df_item_volume['prediction'] = df_item_volume['image_link'].apply(extract_value_unit_from_image)

# Display the updated DataFrame
print(df_item_volume[["index",'prediction']])


/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Error downloading image from https://m.media-amazon.com/images/I/81bLHedrwL._AC_SL1500_.jpg: 400 Client Error: Bad Request for url: https://m.media-amazon.com/images/I/81bLHedrwL._AC_SL1500_.jpg
Error downloading image from https://m.media-amazon.com/images/I/711dIDwDuJ._AC_SL1500_.jpg: 400 Client Error: Bad Request for url: https://m.media-amazon.com/images/I/711dIDwDuJ._AC_SL1500_.jpg
Error downloading image from https://m.media-amazon.com/images/I/51MVfM5NJV._AC_SL1500_.jpg: 400 Client Error: Bad Request for url: https://m.media-amazon.com/images/I/51MVfM5NJV._AC_SL1500_.jpg
    index prediction
0  128522     946 mL
1  128501           
2  115029           
3   41358           
4  112194           


In [ ]:
import requests
from PIL import Image, ImageOps
from io import BytesIO
import numpy as np
import pandas as pd
import easyocr
import re

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)

# Function to preprocess and perform OCR on the image URL
def extract_value_unit_from_image(image_url):
    try:
        # Send a request to get the image from the URL
        response = requests.get(image_url)
        response.raise_for_status()  # Ensure the request was successful

        # Open the image and convert it to grayscale
        img = Image.open(BytesIO(response.content))
        gray_img = ImageOps.grayscale(img)
        
        # Convert Pillow image to numpy array for OCR
        np_image = np.array(gray_img)
        
        # Perform OCR using EasyOCR
        result = reader.readtext(np_image, detail=0)  # detail=0 returns only the text
        extracted_text = " ".join(result)  # Join the OCR result into a single string
        
        # Use regex to find the value + unit (e.g., numbers followed by units like cm³, L, etc.)
        pattern = r'(\d+\.?\d*)\s?(cm³|liters|ml|g|kg|m³|in³|L|oz|fl oz)'  # Modify as per expected units
        matches = re.findall(pattern, extracted_text, re.IGNORECASE)
        
        if matches:
            # Return the first match (value + unit)
            return f"{matches[0][0]} {matches[0][1]}"
        else:
            # Return blank if no match is found
            return " "
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {image_url}: {e}")
        return " "  # Return blank in case of an error

# Create a DataFrame (this is an example, replace it with your actual data loading)
df_item_volume = pd.DataFrame({
    'index': [128522, 128501, 115029, 41358, 112194, 114978, 127232, 127456, 112191],
    'image_link': [
        "https://m.media-amazon.com/images/I/81bgTNB4g-L.jpg",
        "https://m.media-amazon.com/images/I/81bLHedrwL.jpg",
        "https://m.media-amazon.com/images/I/711dIDwDuJ.jpg",
        "https://m.media-amazon.com/images/I/51MVfM5NJV.jpg",
        "https://m.media-amazon.com/images/I/61jCNOENUwL.jpg",
        "https://m.media-amazon.com/images/I/711EbPu-Nh.jpg",
        "https://m.media-amazon.com/images/I/81KOZGbVZB.jpg",
        "https://m.media-amazon.com/images/I/81N6CnmBpr.jpg",
        "https://m.media-amazon.com/images/I/61j9aDDdjD.jpg"
    ],
    'group_id': [257505, 426449, 142295, 588284, 784861, 417442, 125148, 162326, 472418],
    'entity_name': ['item_volume'] * 9
})

# Create a new column to store the extracted item volume (value + unit)
df_item_volume['prediction'] = df_item_volume['image_link'].apply(extract_value_unit_from_image)

# Save the updated DataFrame to a CSV file
output_file = 'item_volume_predictions.csv'
df_item_volume.to_csv(output_file, index=False)

print(f"DataFrame with predictions saved to {output_file}")

# Display the updated DataFrame
print(df_item_volume[["index", "prediction"]])


In [ ]:
import requests
from PIL import Image, ImageOps
from io import BytesIO
import numpy as np
import pandas as pd
import easyocr
import re

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)

# Function to preprocess and perform OCR on the image URL
def extract_value_unit_from_image(image_url):
    try:
        # Send a request to get the image from the URL
        response = requests.get(image_url)
        response.raise_for_status()  # Ensure the request was successful

        # Open the image and convert it to grayscale
        img = Image.open(BytesIO(response.content))
        gray_img = ImageOps.grayscale(img)
        
        # Convert Pillow image to numpy array for OCR
        np_image = np.array(gray_img)
        
        # Perform OCR using EasyOCR
        result = reader.readtext(np_image, detail=0)  # detail=0 returns only the text
        extracted_text = " ".join(result)  # Join the OCR result into a single string
        
        # Use regex to find the value + unit (e.g., numbers followed by units like cm³, L, etc.)
        pattern = r'(\d+\.?\d*)\s?(cm³|liters|ml|g|kg|m³|in³|L|oz|fl oz)'  # Modify as per expected units
        matches = re.findall(pattern, extracted_text, re.IGNORECASE)
        
        if matches:
            # Return the first match (value + unit)
            return f"{matches[0][0]} {matches[0][1]}"
        else:
            # Return blank if no match is found
            return " "
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {image_url}: {e}")
        return " "  # Return blank in case of an error

# DataFrame of image URLs
df_item_volume = pd.DataFrame({
    'index': [128522, 128501, 115029, 41358, 112194, 114978, 127232, 127456, 112191],
    'image_link': [
        "https://m.media-amazon.com/images/I/81bgTNB4g-L._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/81bLHedrwLL._AC_SL1500_.jpg",  # Corrected URL
        "https://m.media-amazon.com/images/I/711dIDwDuJL._AC_SL1500_.jpg",  # Corrected URL
        "https://m.media-amazon.com/images/I/51MVfM5NJVTL._AC_SL1500_.jpg",  # Corrected URL
        "https://m.media-amazon.com/images/I/61jCNOENUwL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/711EbPu-NhLL._AC_SL1500_.jpg",  # Corrected URL
        "https://m.media-amazon.com/images/I/81KOZGbVZBL._AC_SL1500_.jpg",  # Corrected URL
        "https://m.media-amazon.com/images/I/81N6CnmBprL._AC_SL1500_.jpg",  # Corrected URL
        "https://m.media-amazon.com/images/I/61j9aDDdjDL._AC_SL1500_.jpg"   # Corrected URL
    ],
    'group_id': [257505, 426449, 142295, 588284, 784861, 417442, 125148, 162326, 472418],
    'entity_name': ['item_volume'] * 9
})

# Create a new column to store the extracted item volume (value + unit)
df_item_volume['prediction'] = df_item_volume['image_link'].apply(extract_value_unit_from_image)

# Save the updated DataFrame to a CSV file
output_file = 'item_volume_predictions.csv'
df_item_volume.to_csv(output_file, index=False)

print(f"DataFrame with predictions saved to {output_file}")

# Display the updated DataFrame
print(df_item_volume[["index", "prediction"]])


In [ ]:
import requests
from PIL import Image, ImageOps
from io import BytesIO
import numpy as np
import pandas as pd
import easyocr
import re

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)

# Function to preprocess and perform OCR on the image URL
def extract_value_unit_from_image(image_url):
    try:
        # Send a request to get the image from the URL
        response = requests.get(image_url)
        response.raise_for_status()  # Ensure the request was successful

        # Open the image and convert it to grayscale
        img = Image.open(BytesIO(response.content))
        gray_img = ImageOps.grayscale(img)
        
        # Convert Pillow image to numpy array for OCR
        np_image = np.array(gray_img)
        
        # Perform OCR using EasyOCR
        result = reader.readtext(np_image, detail=0)  # detail=0 returns only the text
        extracted_text = " ".join(result)  # Join the OCR result into a single string
        
        # Use regex to find the value + unit (e.g., numbers followed by units like cm³, L, etc.)
        pattern = r'(\d+\.?\d*)\s?(cm³|liters|ml|g|kg|m³|in³|L|oz|fl oz)'  # Modify as per expected units
        matches = re.findall(pattern, extracted_text, re.IGNORECASE)
        
        if matches:
            # Return the first match (value + unit)
            return f"{matches[0][0]} {matches[0][1]}"
        else:
            # Return blank if no match is found
            return " "
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {image_url}: {e}")
        return " "  # Return blank in case of an error

df_item_volume = pd.DataFrame({
    'index': [45081, 127477, 111793, 112234, 114989, 128678, 111771, 121564, 111742, 63256, 112264],
    'image_link': [
        "https://m.media-amazon.com/images/I/51Qb3xsnqjL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/81NLC-MbzNL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61g-R0dUWvL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61jWtTCGOFL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/711JmTEgBUL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/81dup5ucjWL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61fwkjqJ0mL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/71cjjOrq97L._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61fn0eiL6FL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/51kWTG56JdL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61jnmb4d4JL._AC_SL1500_.jpg"
    ],
    'group_id': [961822, 911827, 373285, 588284, 692827, 976920, 941694, 142295, 287137, 941694, 693967],
    'entity_name': ['item_volume'] * 11
})

# Create a new column to store the extracted item volume (value + unit)
df_item_volume['prediction'] = df_item_volume['image_link'].apply(extract_value_unit_from_image)

# Save the updated DataFrame to a CSV file
output_file = 'item_volume_predictions.csv'
df_item_volume.to_csv(output_file, index=False)

print(f"DataFrame with predictions saved to {output_file}")

# Display the updated DataFrame
print(df_item_volume[["index", "prediction"]])


In [ ]:
df_item_volume = pd.DataFrame({
    'index': [45081, 127477, 111793, 112234, 114989, 128678, 111771, 121564, 111742, 63256, 112264],
    'image_link': [
        "https://m.media-amazon.com/images/I/51Qb3xsnqjL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/81NLC-MbzNL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61g-R0dUWvL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61jWtTCGOFL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/711JmTEgBUL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/81dup5ucjWL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61fwkjqJ0mL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/71cjjOrq97L._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61fn0eiL6FL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/51kWTG56JdL._AC_SL1500_.jpg",
        "https://m.media-amazon.com/images/I/61jnmb4d4JL._AC_SL1500_.jpg"
    ],
    'group_id': [961822, 911827, 373285, 588284, 692827, 976920, 941694, 142295, 287137, 941694, 693967],
    'entity_name': ['item_volume'] * 11
})

# GPU100 ACCELERETOR

> Explanation

# Load Dataset with Index: 
* When reading the CSV, index_col=0 ensures that the first column of your CSV is used as the DataFrame index. Adjust index_col based on the actual column in your CSV that contains the index.

# Extract Volume: 
* The extract_value_unit_from_image function is applied to each image URL.

# Create Output DataFrame:
* The new DataFrame output_df contains only the prediction column and retains the original index from your dataset.

In [ ]:
import pandas as pd
import re
import requests
from PIL import Image, ImageOps
from io import BytesIO
import numpy as np
import easyocr

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)

def extract_volume(text):
    # Improved regular expression pattern for item volumes
    volume_pattern = r"(\d+\.?\d*)\s?(mL|ml|liters|L|fl oz|FL 0z|FL oz|oz|cm³|in³|cubic inches|cubic cm)"
    
    # Search for all matches in the text
    matches = re.findall(volume_pattern, text, re.IGNORECASE)
    
    # Prepare a list to store formatted volumes
    volumes = []
    
    # Process each match to clean up the format
    for match in matches:
        volume_value, volume_unit = match
        volume_value = volume_value.strip()  # Remove extra spaces
        volume_unit = volume_unit.lower()    # Normalize units to lower case
        volumes.append(f"{volume_value} {volume_unit}")

    # Return list of extracted volumes
    return volumes

def extract_value_unit_from_image(image_url):
    try:
        # Download and preprocess the image
        response = requests.get(image_url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        gray_img = ImageOps.grayscale(img)
        
        # Convert Pillow image to numpy array
        np_image = np.array(gray_img)
        
        # Perform OCR using EasyOCR
        result = reader.readtext(np_image, detail=0)  # detail=0 returns only the text
        extracted_text = " ".join(result)
        
        # Extract volumes from the OCR result
        volumes = extract_volume(extracted_text)
        return volumes
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {image_url}: {e}")
        return []

# Load your full dataset
df_item_volume = pd.read_csv('/kaggle/working/item_volume_data.csv', index_col=0)  # Assuming the index column is the first column

# Apply the function to the 'image_link' column
df_item_volume['prediction'] = df_item_volume['image_link'].apply(extract_value_unit_from_image)

# Create a new DataFrame with only the index and prediction columns
output_df = df_item_volume[['prediction']].copy()
output_df.index.name = 'index'  # Ensure the index name is set

# Save the updated DataFrame with predictions to a new CSV file
output_df.to_csv('/kaggle/working/item_volume_predictions_final.csv', index=True)

# Display the head of the updated DataFrame to confirm predictions
print(output_df.head())


/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [ ]:
item_volume.tail()

In [ ]:
import requests
from PIL import Image, ImageOps
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import easyocr

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)

# Sample image URL
image_url = "https://m.media-amazon.com/images/I/61jWtTCGOFL._AC_SL1500_.jpg"

# Function to preprocess and display the image
def preprocess_image(image_url):
    try:
        # Send a request to get the image from the URL
        response = requests.get(image_url)
        response.raise_for_status()  # Ensure the request was successful
        
        # Open the image and convert it to grayscale
        img = Image.open(BytesIO(response.content))
        gray_img = ImageOps.grayscale(img)
        
        # Display the image
        plt.imshow(gray_img, cmap='gray')
        plt.axis('off')  # Turn off the axis
        plt.show()
        
        return gray_img
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {image_url}: {e}")
        return None

# Preprocess and display the image
preprocessed_image = preprocess_image(image_url)

# Perform OCR if the image was processed successfully
if preprocessed_image:
    # Convert Pillow image to numpy array
    np_image = np.array(preprocessed_image)
    
    # Perform OCR using EasyOCR
    result = reader.readtext(np_image, detail=0)  # detail=0 returns only the text
    extracted_text = " ".join(result)  # Join the OCR result into a single string
    print(extracted_text)
else:
    print("Error processing image")


In [ ]:
import re

def extract_volume(text):
    # Regular expression pattern to match item volumes (e.g., "32 fl oz", "946 mL", "1.5 liters", "250 cm³")
    volume_pattern = r"(\d+\.?\d*)\s?(mL|ml|liters|L|fl oz|FL 0z|oz|cm³|in³|cubic inches|cubic cm)"
    
    # Search for all matches in the text
    matches = re.findall(volume_pattern, text, re.IGNORECASE)
    
    # Prepare a list to store formatted volumes
    volumes = []
    
    # Process each match to clean up the format
    for match in matches:
        volume_value, volume_unit = match
        volume_value = volume_value.strip()  # Remove extra spaces
        volume_unit = volume_unit.lower()    # Normalize units to lower case
        volumes.append(f"{volume_value} {volume_unit}")

    # Return list of extracted volumes
    return volumes

import re

text = "& AX VS 4X 'OKupoarduow 6 4 ML) [6len (946 Foncs POND Mirkqe Scrnces CLEANER AX MUCK  MORE NATURAL 'REMOVER CONCENTRATED SLUDGE / TREATS L AwakaT 32FL 0z Uerkhoia Jemu FOR SAFEF"
pattern = r'(\d+)\s*(FL 0z)'
matches = re.findall(pattern, text)

volumes = [f"{match[0]} {match[1]}" for match in matches]
print("Extracted Volumes:", volumes)


In [ ]:
import re

def extract_total_ml(text):
    # Regular expression to find values in various formats
    pattern = r'\b\d+(\.\d+)?\s?ml\b'
    matches = re.findall(pattern, text, re.IGNORECASE)
    
    # Convert matches to float and sum them up
    total_volume = sum(float(match.replace(',', '.')) for match in matches)
    
    return f"{total_volume:.1f} ml"

# Example text
text = """FOOD GRADE VEGAN MSDS AOCOCOLORING Fu_COLORING HCOCOLORING OREEN Red SkY BLUE MWT: IOML (0.35021 | FMT: TOML (0,3502) FFTWT: IOML (0,3502) FLdD COLORING ORING XUCOLORLNG '6UCOLORING MSDS MSDS MSDS FLI DCOLORING FcJO COLORING HAvy BLUE KSUNSET VELLOH RASS CREEN PINK BROWN VETMT: TOML (0.3502] IOML (0,3502) JM: TOML (0.3507) Lik IOML (0.3502] IETWT: IOML (0,35021 DS MSDS 50s 2 1 3 3 1 Gluten Free Nut Free Sugar Free Egg Free Dairy Free Free 8 1 I 1 { 0 ( 0 { 8 Soy"""

# Extract total ml
total_ml = extract_total_ml(text)
print(total_ml)


In [ ]:
import pandas as pd

# Load the 'depth_data.csv' file
item_volume_df = pd.read_csv('item_volume_data.csv')

# Display the top 5 rows of the 'depth_data.csv'
print(item_volume_df.head(100))


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/kaggle/input/real-test-data/dataset/test.csv')

# Create a new DataFrame with only the 'index' column
df_new = df[['index']].copy()

# Add a new column 'prediction' with blank values
df_new['prediction'] = " "  # Ensure all values are blank

# Optionally, save the new DataFrame to a new CSV file
df_new.to_csv('submission.csv', index=False)


In [ ]:
df_new.tail()

In [ ]:
ss=pd.read_csv("/kaggle/input/real-test-data/dataset/test.csv")
ss.tail()